# Plotting using bqplot

Works only when you install jupyterlab in the same environment as you install bqplot, and launch a notebook from this environment

Otherwise, it gives an error along the lines of 'model not found', or returns the location of the plot object with

Run these commands:

source envs/bqplot-env/bin/activate

jupyter lab --no-browser --ip=$(hostname --fqdn)

In [1]:
import numpy as np
from bqplot import pyplot as plt

In [2]:
plt.figure(1, title='Line Chart')
np.random.seed(0)
n = 200
x = np.linspace(0.0, 10.0, n)
y = np.cumsum(np.random.randn(n))
plt.plot(x, y)
plt.show()

In [1]:
import numpy as np
from bqplot import (
    OrdinalScale, LinearScale, Bars, Lines, Axis, Figure
)

In [2]:
size = 20
np.random.seed(0)

x_data = np.arange(size)

x_ord = OrdinalScale()
y_sc = LinearScale()

bar = Bars(x=x_data, y=np.random.randn(2, size), scales={'x': x_ord, 'y':
y_sc}, type='stacked')
line = Lines(x=x_data, y=np.random.randn(size), scales={'x': x_ord, 'y': y_sc},
             stroke_width=3, colors=['red'], display_legend=True, labels=['Line chart'])

ax_x = Axis(scale=x_ord, grid_lines='solid', label='X')
ax_y = Axis(scale=y_sc, orientation='vertical', tick_format='0.2f',
            grid_lines='solid', label='Y')

Figure(marks=[bar, line], axes=[ax_x, ax_y], title='API Example',
       legend_location='bottom-right')

Figure(axes=[Axis(label='X', scale=OrdinalScale()), Axis(label='Y', orientation='vertical', scale=LinearScale(…

In [3]:
from bqplot import *
from IPython.display import display

x_data = range(10)
y_data = [i ** 2 for i in x_data]

x_sc = LinearScale()
y_sc = LinearScale()

ax_x = Axis(label='Test X', scale=x_sc, tick_format='0.0f')
ax_y = Axis(label='Test Y', scale=y_sc,
            orientation='vertical', tick_format='0.2f')

line = Lines(x=x_data,
             y=y_data,
             scales={'x': x_sc, 'y': y_sc},
             colors=['red', 'yellow'])

fig = Figure(axes=[ax_x, ax_y], marks=[line])

display(fig)


Figure(axes=[Axis(label='Test X', scale=LinearScale(), tick_format='0.0f'), Axis(label='Test Y', orientation='…

# Plot SNNS vs Keras scatterplot

In [1]:
import pandas as pd
from bqplot import (
    LinearScale, Scatter, Lines, Figure, Axis, pyplot as plt,
)
from ipywidgets import Output

In [5]:
d_path='/cluster/gjb_lab/2472402/data/dssp_dict.csv'
seq_dict=dict()
with open(d_path,'r') as f:
    rows=f.read().splitlines()
    while (rows):
        row=rows.pop(0)
        domain, seqID = row.split(',')
        seq_dict[int(seqID)]=domain

In [2]:
i=1

In [3]:
keras_path="/cluster/gjb_lab/2472402/results/keras/25Aug/cv%d_scores.csv" % i
snns_path="/cluster/gjb_lab/2472402/results/snns/25Aug/cv%d_scores.csv" % i
snns=pd.read_csv(snns_path)
keras=pd.read_csv(keras_path)
seqIDs=snns.seqID
snns=snns.set_index('seqID')
keras=keras.set_index('seqID')

In [4]:
keras_path_2="/cluster/gjb_lab/2472402/results/keras/25Aug/cv%d.knet" % i
snns_path_2="/cluster/gjb_lab/2472402/results/snns/25Aug/cv%d.knet" % i
dssp=[]
snns_pred=[]
keras_pred=[]

# accumulate the 3 lists above
with open(keras_path_2) as f:
    rows=f.read().splitlines()
    while (rows):
        row=rows.pop(0)
        if (row.startswith('DSSP')):
            dssp.append(row[11:])
        elif (row.startswith('HMM_pred')):
            keras_pred.append(row[11:])
        else:
            continue

with open(snns_path_2) as f:
    rows=f.read().splitlines()
    while (rows):
        row=rows.pop(0)
        if (row.startswith('HMM_pred')):
            snns_pred.append(row[11:])
        else:
            continue

In [10]:
dpred=dict()
for (j,d,s,k) in zip (seqIDs,dssp,snns_pred,keras_pred):
    dpred[j]={'domain':seq_dict[j],'dssp_str':d,'snns_str':s,'keras_str':k}

In [9]:
x_data=snns.HMM_acc
y_data=keras.HMM_acc

x_sc=LinearScale()
y_sc=LinearScale()

ax_x = Axis(label='SNNS HMM acc', scale=x_sc, tick_format='0.1f')
ax_y = Axis(label='Keras HMM acc', scale=y_sc, orientation='vertical', tick_format='0.1f')

# create Output widget
out=Output() 

# define function to run when I hover over a mark
def def_tt(_,event):
    out.clear_output()
    with out:
        seqID=event['data']['name']
        val_x=event['data']['x']
        val_y=event['data']['y']
        domain=dpred[seqID]['domain']
        dssp_str=dpred[seqID]['dssp_str']
        snns_str=dpred[seqID]['snns_str']
        keras_str=dpred[seqID]['keras_str']
        print('seqID:  %s' % seqID)
        print('SCOPe:  %s' % domain)
        print('snns:   %s' % val_x)
        print('keras:  %s' % val_y)
        print('dssp:   %s' % dssp_str)
        print('snns:   %s' % snns_str)
        print('keras:  %s' % keras_str)

scatter=Scatter(x=x_data,
                y=y_data,
                scales={'x': x_sc, 'y': y_sc},
                names=seqIDs,
                display_names=False,
                tooltip=out,
                opacities=[0.5],
                hovered_style={'opacity':1.0,'fill':'DarkOrange'},
                unhovered_style={'opacity':0.2,},
               )

scatter.on_hover(def_tt)

line=Lines(x=[60,100],y=[60,100],
           scales={'x': x_sc, 'y': y_sc},
           colors=['White'],
           stroke_width=0.5,
           line_style='dashed'
          )

Figure(title='Cross-validation fold %d accuracies' % i,
       marks=[scatter,line],
       axes=[ax_x,ax_y],
       layout={'height':'900px','width':'900px'},
      )

Figure(axes=[Axis(label='SNNS HMM acc', scale=LinearScale(), tick_format='0.1f'), Axis(label='Keras HMM acc', …